In [2]:
#import library
import os
import sys
import ta
import nltk
import talib
import pynance
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

Load dataset

In [3]:
#load news analys rating dataset
news_data = pd.read_csv("../data/raw_analyst_ratings.csv")
news_data.head()

,Unnamed: 0,headline,url,publisher,date,stock
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A


In [4]:
#load sock dataset
stock_data = pd.read_csv("../data/AAPL_historical_data.csv")
stock_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.098943,469033600,0.0,0.0
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.093781,175884800,0.0,0.0
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.086898,105728000,0.0,0.0
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089049,86441600,0.0,0.0
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.091630,73449600,0.0,0.0


Converting data to standard format

In [5]:
#converting news data to standard date format
news_data['date'] = news_data['date'].str.slice(0, 19)   
# Convert the 'date' column to datetime format
news_data['date'] = pd.to_datetime(news_data['date']).dt.date

In [6]:
#Converting stock date to standared date format
stock_data['Date'] = pd.to_datetime(stock_data['Date']).dt.date

In [7]:
# Filter stock data to match news data date range
start_date = news_data['date'].min()
end_date = news_data['date'].max()
stock_data = stock_data[(stock_data['Date'] >= start_date) & (stock_data['Date'] <= end_date)]

stock_data.head()


,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
7109,2009-02-17,3.459643,3.465714,3.367143,3.376071,2.851008,678238400,0.0,0.0
7110,2009-02-18,3.394643,3.423214,3.311429,3.370357,2.846183,684779200,0.0,0.0
7111,2009-02-19,3.334643,3.366071,3.218214,3.237143,2.733687,922804400,0.0,0.0
7112,2009-02-20,3.192857,3.300000,3.178571,3.257143,2.750577,750316000,0.0,0.0
7113,2009-02-23,3.273214,3.285714,3.089643,3.105357,2.622398,786982000,0.0,0.0


In [8]:
news_data.head()

,Unnamed: 0,headline,url,publisher,date,stock
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03,A
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26,A
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22,A
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22,A


In [9]:
# Merge datasets on date
merged_data = pd.merge(news_data, stock_data, left_on='date', right_on='Date', how='inner')

# Display the number of matched rows
print(f'Number of matched rows: {len(merged_data)}')

# Optional: Display some matched data for verification
merged_data.head()

Number of matched rows: 1379646


,Unnamed: 0,headline,url,publisher,date,stock,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A,2020-06-05,80.837502,82.937500,80.807503,82.875000,80.843407,137250400,0.0,0.0
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03,A,2020-06-03,81.165001,81.550003,80.574997,81.279999,79.287506,104491200,0.0,0.0
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26,A,2020-05-26,80.875000,81.059998,79.125000,79.182503,77.241432,125522000,0.0,0.0
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22,A,2020-05-22,78.942497,79.807503,78.837502,79.722504,77.768188,81803200,0.0,0.0
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22,A,2020-05-22,78.942497,79.807503,78.837502,79.722504,77.768188,81803200,0.0,0.0


In [10]:
merged_data.shape

(1379646, 15)

Data cleaning

In [11]:
#drop unamed columns
merged_data.drop(columns=['Unnamed: 0'], inplace=True)

# Drop one of the duplicate date columns, let's drop 'date' and keep 'Date'
merged_data.drop(columns=['date'], inplace=True)

In [12]:
merged_data.head()

,headline,url,publisher,stock,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,A,2020-06-05,80.837502,82.937500,80.807503,82.875000,80.843407,137250400,0.0,0.0
1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,A,2020-06-03,81.165001,81.550003,80.574997,81.279999,79.287506,104491200,0.0,0.0
2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,A,2020-05-26,80.875000,81.059998,79.125000,79.182503,77.241432,125522000,0.0,0.0
3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,A,2020-05-22,78.942497,79.807503,78.837502,79.722504,77.768188,81803200,0.0,0.0
4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,A,2020-05-22,78.942497,79.807503,78.837502,79.722504,77.768188,81803200,0.0,0.0


In [13]:
#Copy merged_data to a new DataFrame with an appropriate name
newsStockData = merged_data.copy()